In [ ]:
#|default_exp foundation

In [ ]:
#|export
from fastcore.imports import *
from fastcore.basics import *
from functools import lru_cache
from contextlib import contextmanager
from copy import copy
from configparser import ConfigParser
import random,pickle,inspect

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *
from fastcore.nb_imports import *

# Foundation

> The `L` class and helpers for it

## Foundational Functions

In [ ]:
#|export
@contextmanager
def working_directory(path):
    "Change working directory to `path` and return to previous on exit."
    prev_cwd = Path.cwd()
    os.chdir(path)
    try: yield
    finally: os.chdir(prev_cwd)

In [ ]:
#|export
def add_docs(cls, cls_doc=None, **docs):
    "Copy values from `docs` to `cls` docstrings, and confirm all public methods are documented"
    if cls_doc is not None: cls.__doc__ = cls_doc
    for k, v in docs.items():
        f = getattr(cls, k)
        if hasattr(f, '__func__'): f = f.__func__ # required for class methods
        f.__doc__ = v
        
    # List of public callables without docstring
    nodoc = [c for n, c in vars(cls).items() if callable(c)
             and not n.startswith('_') and c.__doc__ is None and c.__doc__ is None]
    assert not nodoc, f"Missing docs: {nodoc}"
    assert cls.__doc__ is not None, f"Missing class docs: {cls}"

In [ ]:
class _T:
    def __init__(self, a, b):
        self.a = a
        self.b = b
a = _T(1,2)
vars(a).items()

dict_items([('a', 1), ('b', 2)])

`add_docs` allows you to add docstrings to a class and its associated methods. This function allows you to group docstrings together seperate from your code, which enables you to define one-line functions as well as organize your code more succintly. We believe this confers a number of benefits which we discuss in our style guide.

Suppose you have the following undocumented class:

In [ ]:
class T:
    def foo(self): pass
    def bar(self): pass

You can add documentation to this class like so:

In [ ]:
add_docs(T, cls_doc="A docstring for the class.",
            foo="The foo method.",
            bar="The bar method.")

In [ ]:
test_eq(T.__doc__, "A docstring for the class.")
test_eq(T.foo.__doc__, "The foo method.")
test_eq(T.bar.__doc__, "The bar method.")

`add_docs` also validates that all of  your public methods contain a docstring.  If one of your methods is not documented, it will raise an error:

In [ ]:
class T:
    def foo(self): pass
    def bar(self): pass

f=lambda: add_docs(T, "A docstring for the class.", foo="The foo method.")
test_fail(f, contains="Missing docs")

In [ ]:
#|hide
class _T:
    def f(self): pass
    @classmethod
    def g(cls): pass
add_docs(_T, "a", f="f", g="g")

test_eq(_T.__doc__, "a")
test_eq(_T.f.__doc__, "f")
test_eq(_T.g.__doc__,"g")

In [ ]:
#|export
def docs(cls):
    "Decorator version of `add_docs`, using `_docs` dict"
    add_docs(cls, **cls._docs)
    return cls

Instead of using `add_docs`, you can use the decorator `docs` as shown below.  Note that the docstring for the class can be set with the argument `cls_doc`:

In [ ]:
@docs
class _T:
    def f(self): pass
    def g(self): pass
    
    _docs = dict(cls_doc="The class docstring",
                 f="The docstring for method f",
                 g="A different docstring for method g.")
    
test_eq(_T.__doc__, "The class docstring")
test_eq(_T.f.__doc__, "The docstring for method f")
test_eq(_T.g.__doc__,"A different docstring for method g.")

For either the `docs` decorator or the `add_docs` function, you can still define your docstrings in the normal way.  Below we set the docstring for the class as usual, but define the method docstrings through the `_docs` attribute:

In [ ]:
@docs
class _T:
    "The class docstring"
    def f(self): pass
    _docs = dict(f="The docstring for method f.")

    
test_eq(_T.__doc__, "The class docstring")
test_eq(_T.f.__doc__, "The docstring for method f.")

In [ ]:
show_doc(is_iter)

---

### is_iter

>      is_iter (o)

Test whether `o` can be used in a `for` loop

In [ ]:
assert is_iter([1])
assert not is_iter(array(1))
assert is_iter(array([1,2]))
assert (o for o in range(3))

In [ ]:
#| export
def coll_repr(c, max_n=10):
    "String repr of up to `max_n` items of (possibly lazy) collection `c`"
    return f'(#{len(c)}) [' + ','.join(itertools.islice(map(repr,c), max_n)) + (
        '...' if len(c) > max_n else '') + ']'

In [ ]:
#| hide
it = (o for o in range(100))
list(itertools.islice(map(repr,it), 10))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

`coll_repr` is used to provide a more informative [`__repr__`](https://stackoverflow.com/questions/1984162/purpose-of-pythons-repr) about list-like objects.  `coll_repr` is used by `L` to build a `__repr__` that displays the length of a list in addition to a preview of a list.

Below is an example of the `__repr__` string created for a list of 1000 elements:

In [ ]:
test_eq(coll_repr(range(1000)),    '(#1000) [0,1,2,3,4,5,6,7,8,9...]')
test_eq(coll_repr(range(1000), 5), '(#1000) [0,1,2,3,4...]')
test_eq(coll_repr(range(10),   5),   '(#10) [0,1,2,3,4...]')
test_eq(coll_repr(range(5),    5),    '(#5) [0,1,2,3,4]')

We can set the option `max_n` to optionally preview a specified number of items instead of the default:

In [ ]:
test_eq(coll_repr(range(1000), max_n=5), '(#1000) [0,1,2,3,4...]')

In [ ]:
#|export
def is_bool(x):
    "Check whether `x` is a bool or None"
    return isinstance(x, (bool, NoneType)) or risinstance('bool_', x)

In [ ]:
#|export
def mask2idxs(mask):
    "Convert bool mask or index list to index `L`"
    if isinstance(mask, slice): return mask
    mask = list(mask)
    if len(mask)==0: return []
    it = mask[0]
    # next line is not entirely clear for me
    # if hasattr(it, 'item'): it = it.item()
    if is_bool(it): return [i for i,m in enumerate(mask) if m]
    return [int(i) for i in mask]

In [ ]:
test_eq(mask2idxs([False,True,False,True]), [1,3])
test_eq(mask2idxs(array([False,True,False,True])), [1,3])
test_eq(mask2idxs(array([1,2,3])), [1,2,3])

In [ ]:
#|export
def cycle(o):
    "Like `itertools.cycle` except creates list of `None`s if `o` is empty"
    o = listify(o)
    return itertools.cycle(o) if o is not None and len(o) > 0 else itertools.cycle([None])

In [ ]:
test_eq(itertools.islice(cycle([1,2,3]),5), [1,2,3,1,2])
test_eq(itertools.islice(cycle([]),3), [None]*3)
test_eq(itertools.islice(cycle(None),3), [None]*3)
test_eq(itertools.islice(cycle(1),3), [1,1,1])

In [ ]:
#|export
def zip_cycle(x, *args):
    "Like `itertools.zip_longest` but `cycle`s through elements of all but first argument"
    return zip(x, *map(cycle, args))

In [ ]:
#|hide
list(itertools.zip_longest([1,2,3,4],list('abc')))
#list(itertools.zip_longest([1,2],list('abc')))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, None)]

In [ ]:
test_eq(zip_cycle([1,2,3,4],list('abc')), [(1, 'a'), (2, 'b'), (3, 'c'), (4, 'a')])
test_eq(zip_cycle([1,2],list('abc')), [(1, 'a'), (2, 'b')])

In [ ]:
#|export
def is_indexer(idx):
    """Test whether `idx` will index a single item in a list. 
    if `getattr` returns `ndim` >0, it will not index into single item in a list -> False"""
    return isinstance(idx, int) or not getattr(idx, 'ndim', 1)

In [ ]:
#|hide
test_eq(not getattr(array(2), 'ndim', 1), True)
test_eq(not getattr(array([1,2]), 'ndim', 1), False)

You can, for example index a single item in a list with an integer or a 0-dimensional numpy array:

In [ ]:
assert is_indexer(1)
assert is_indexer(np.array(1))

However, you cannot index into single item in a list with another list or a numpy array with ndim > 0. 

In [ ]:
assert not is_indexer([1, 2])
assert not is_indexer(np.array([[1, 2], [3, 4]]))

## `L` helpers

In [ ]:
#|export
class CollBase:
    "Base class for composing a list of `items`"
    def __init__(self, items): self.items = items
    def __len__(self): return len(self.items)
    def __getitem__(self, k): return self.items[list(k) if isinstance(k, CollBase) else k]
    def __setitem__(self, k, v): self.items[list(k) if isinstance(k, CollBase) else k] = v
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self): return self.items.__repr__()
    def __iter__(self): return self.items.__iter__()

`ColBase` is a base class that emulates the functionality of a python `list`:

In [ ]:
class _T(CollBase): pass
l = _T([1,2,3,4,5])

test_eq(len(l), 5) # __len__
test_eq(l[-1], 5); test_eq(l[0], 1) #__getitem__
l[2] = 100; test_eq(l[2], 100)      # __set_item__
del l[0]; test_eq(len(l), 4)        # __delitem__
test_eq(str(l), '[2, 100, 4, 5]')   # __repr__

## L -

Note to self: I will try to tackle that class piece by piece as some parts (MetaClass and GetAttr) are unclear to me now

In [ ]:
#| export
class L(GetAttr, CollBase):#, metaclass=_L_Meta):
    "Behaves like a list of `items` but can also index with list of indices or masks"
    _default = 'items'
    def __init__(self, items=None, *rest, use_list=False, match=None):
        if (use_list is not None) or not is_array(items):
            items = listify(items, *rest, use_list=use_list, match=match)
            super().__init__(items)
    
    @property
    def _xtra(self): return None
    def _new(self, items, *args, **kwargs): return type(self)(items, *args, use_list=False, **kwargs)
    # if indexing with idx does not return a single item wrap return value into L
    def __getitem__(self, idx): return self._get(idx) if is_indexer(idx) else L(self._get(idx), use_list=None)
    def copy(self): return _new(self.items.copy())

    def _get(self, i):
        # getattr tries iloc on self.items if they are of pd dataseries type
        if is_indexer(i) or isinstance(i, slice): return getattr(self.items, 'iloc', self.items)[i]
        i = mask2idxs(i)
        return (self.items.iloc[list(i)] if hasattr(self.items, 'iloc')
                else self.items.__array__()[(i,)] if hasattr(self.items, '__array__')
                else [self.items[i_] for i_ in i])
    
    def __setitem__(self, idx, o):
        "Set `idx` (can be list of indices, or mask, or int) items to `o` (which is broadcast if not iterable)"
        if isinstance(idx, int): self.items[idx] = o
        else:
            idx = idx if isinstance(idx, L) else listify(idx)
            if not is_iter(o): o = [o]*len(idx)
            for i,o_ in zip(idx,o): self.items[i] = o_
            
    def __eq__(self, b):
        if b is None: return False
        # array_equal will compare `__array__`
        if risinstance('ndarray', b): return array_equal(b, self)
        if isinstance(b, (str,dict)) or callable(b): return False
        return all_equal(b, self)
    
    # TODO: check sorted_ex
    def sorted(self, key=None, reverse=False): return self._new(sorted_ex(self, key=key, reverse=reverse))
    def __iter__(self): return iter(self.items.itertuples() if hasattr(self.items, 'iloc') else self.items)
    def __contains__(self, b): return b in self.items
    def __reversed__(self): self._new(reversed(self.items))
    # TODO: check later
    def __invert__(self): self._new(not i for i in self)
    def __repr__(self): return repr(self.items)
    # TODO: check later
    def _repr_pretty_(self, p, cycle):
        p.text('...' if cycle else repr(self.items) if is_array(self.items) else coll_repr(self))
        
    def __mul__ (a,b): return a._new(a.items*b)
    def __add__ (a,b): return a._new(a.items + listify(b)) 
    def __radd__(a,b): return a._new(b)+a
    # ?
    def __addi__(a,b): 
        a.items += list(b)
        return a
    
    @classmethod
    def split(cls, s, sep=None, maxsplit=-1): return cls(s.split(sep, maxsplit))
    @classmethod
    def range(cls, a, b=None, step=None): return cls(range_of(a, b=b, step=step))

    def map(self, f, *args, **kwargs): pass
    def argwhere(self, f, negate=False, **kwargs): pass
    def argfirst(self, f, negate=False) :pass
    def filter(self, f=noop, negate=False, **kwargs):pass

    def enumerate(self): pass
    def renumerate(self): pass
    def unique(self, sort=False, bidir=False, start=None): pass
    def val2idx(self): pass
    def cycle(self): pass
    def map_dict(self, f=noop, *args, **kwargs): pass
    def map_first(self, f=noop, g=noop, *args, **kwargs):pass
       

    def itemgot(self, *idxs): pass
    def attrgot(self, k, default=None): pass

    def starmap(self, f, *args, **kwargs): pass
    def zip(self, cycled=False): pass
    def zipwith(self, *rest, cycled=False): pass
    def map_zip(self, f, *args, cycled=False, **kwargs): pass
    def map_zipwith(self, f, *rest, cycled=False, **kwargs): pass
    def shuffle(self):pass

    def concat(self): pass
    def reduce(self, f, initial=None): pass
    def sum(self): pass
    def product(self): pass
    def setattrs(self, attr, val): pass

In [ ]:
#|export
add_docs(L,
         __getitem__="Retrieve `idx` (can be list of indices, or mask, or int) items",
         range="Class Method: Same as `range`, but returns `L`. Can pass collection for `a`, to use `len(a)`",
         split="Class Method: Same as `str.split`, but returns an `L`",
         copy="Same as `list.copy`, but returns an `L`",
         sorted="New `L` sorted by `key`. If key is str use `attrgetter`; if int use `itemgetter`",
         unique="Unique items, in stable order",
         val2idx="Dict from value to index",
         filter="Create new `L` filtered by predicate `f`, passing `args` and `kwargs` to `f`",
         argwhere="Like `filter`, but return indices for matching items",
         argfirst="Return index of first matching item",
         map="Create new `L` with `f` applied to all `items`, passing `args` and `kwargs` to `f`",
         map_first="First element of `map_filter`",
         map_dict="Like `map`, but creates a dict from `items` to function results",
         starmap="Like `map`, but use `itertools.starmap`",
         itemgot="Create new `L` with item `idx` of all `items`",
         attrgot="Create new `L` with attr `k` (or value `k` for dicts) of all `items`.",
         cycle="Same as `itertools.cycle`",
         enumerate="Same as `enumerate`",
         renumerate="Same as `renumerate`",
         zip="Create new `L` with `zip(*items)`",
         zipwith="Create new `L` with `self` zip with each of `*rest`",
         map_zip="Combine `zip` and `starmap`",
         map_zipwith="Combine `zipwith` and `starmap`",
         concat="Concatenate all elements of list",
         shuffle="Same as `random.shuffle`, but not inplace",
         reduce="Wrapper for `functools.reduce`",
         sum="Sum of the items",
         product="Product of the items",
         setattrs="Call `setattr` on all items"
        )

By calling Sequence.register(ClassName), you are declaring that ClassName should be considered a virtual subclass of Sequence. This does not enforce any checks or require ClassName to implement any of the methods defined by Sequence; it merely updates the internal subclass check mechanism.

In [ ]:
#|export
Sequence.register(L);

`L` is a drop in replacement for a python `list`.  Inspired by [NumPy](http://www.numpy.org/), `L`,  supports advanced indexing and has additional methods (outlined below) that provide additional functionality and encourage simple expressive code. For example, the code below takes a list of pairs, selects the second item of each pair, takes its absolute value, filters items greater than 4, and adds them up:

In [ ]:
# ?? replicate
from fastcore.utils import gt

In [ ]:
gt??

Signature: gt(a, b=<object object at 0x10497b990>)
Docstring: Same as `operator.gt`, or returns partial if 1 arg
Source:        def _inner(a, b=_dumobj): return _oper(op, a,b)
File:      ~/mambaforge/lib/python3.10/site-packages/fastcore/basics.py
Type:      function


In [ ]:
#|hide
#|eval: false
from nbdev import nbdev_export
nbdev_export()